# Record linkage con Python Record Linkage Toolkit

In [ ]:
import recordlinkage
import pandas as pd
import time
start = time.time()

### Pulizia dei dati e modifica campi in maiuscolo

In [ ]:
def getCompleteDataset(path = 'output_alignment_multithread_finalpass/schema_mediato.json'):
    #schema mediato finale di schema alignment
    df = pd.read_json(path)

    df = df[df.name.isnull() == False]

    df['name'] = df['name'].replace(r'\s+|\\n|\\r', ' ', regex=True)
    df['name'] = df['name'].str.upper()
    df['industry'] = df['industry'].str.upper()
    df['country'] = df['country'].str.upper()
    df['headquarters'] = df['headquarters'].str.upper()
    df['address'] = df['address'].str.upper()
    df['sector'] = df['sector'].str.upper()
    df['ceo'] = df['ceo'].str.upper()
    df['founders'] = df['founders'].str.upper()
    df['area_served'] = df['area_served'].str.upper()

    return df

df=getCompleteDataset()

df_a = df
df_b = df

### Blocking sul name

In [ ]:
def block(field, df_a, df_b):
    indexer = recordlinkage.Index()
    indexer.block(field)
    return indexer.index(df_a, df_b)
candidate_links = block('name', df_a, df_b)

### Eliminare coppie uguali e speculari, ad esempio, (A,A) o in casi in cui (A,B) e (B,A)

In [ ]:
def getMultiIndexNoCopy(candidate_links):
    # eliminazione coppie indici uguali, sx-dx = dx-sx
    set_no_copy = set()
    for (c,b) in candidate_links:
        if (b,c) not in set_no_copy and (c != b):
            set_no_copy.add((c,b))

    print("Match trovati senza copie: " + str(len(set_no_copy)))

    list_no_copy = list(set_no_copy)
    # gli elementi delle coppie vengono distribuite su due liste parallele
    list_0 = [x[0] for x in list_no_copy] #indici sinistri
    list_1 = [x[1] for x in list_no_copy] #indici destri

    return pd.MultiIndex.from_arrays([list_0, list_1])
multi_index = getMultiIndexNoCopy(candidate_links)

### Record linkage su (name, name), (country, country), (headquarters, country)

In [ ]:
compare = recordlinkage.Compare()

compare.string('name', 'name', method='jarowinkler', threshold=0.7)
# compare.string('industry', 'industry', method='jarowinkler', threshold=0.85)
compare.string('country', 'country', method='jarowinkler', threshold=0.5, missing_value=1)
compare.string('headquarters', 'country', method='jarowinkler', threshold=0.5, missing_value=1)
# compare.string('country', 'headquarters', method='jarowinkler', threshold=0.5)
# compare.string('headquarters', 'headquarters', method='jarowinkler', threshold=0.5)


# compare.string('headquarters', 'headquarters', method='jarowinkler', threshold=0.85)
# compare.string('ceo', 'ceo', method='jarowinkler', threshold=0.85)
# compare.string('sector', 'sector', method='jarowinkler', threshold=0.85)

# The comparison vectors
compare_vectors = compare.compute(candidate_links, df_a, df_b)
# pulizia dalle coppie in eccesso
compare_vectors = compare_vectors[compare_vectors.index.isin(multi_index)]

### Predizione Record Linkage con ECM Classifier

In [ ]:
def ecmFitPredict(compare_vectors):
    ecm = recordlinkage.ECMClassifier()
    return ecm.fit_predict(compare_vectors)
matches = ecmFitPredict(compare_vectors)

# Creazione di un dizionario con chiave che corrispondono a record relativi ad una entità e come valore una lista di entità che corrispondono a quella relativa alla chiave associata

In [ ]:
def getDictMatches(matches):
    dict_matches = {}
    keyToIgnore = set()
    for k, v in matches:
        if k in keyToIgnore:
            continue
        elif k in dict_matches.keys():
            dict_matches[k].append(v)
            keyToIgnore.add(v)
        else :
            dict_matches[k] = [v]
            keyToIgnore.add(v)
    return dict_matches

dict_matches_global = getDictMatches(matches)

### Vengono collassati i record relativi alla stessa entità in un unico record combinando i valori degli attributi di tutti i record della stessa entità (il valore di ogni attributo di un record collassato corrisponde al primo individuato con valore diverso dal None)

In [ ]:

def collapseMatches(df, dict_matches):
    df_collapsed = pd.DataFrame(columns=['name' , 'industry', 'market_cap', 'employees', 'country', 'headquarters', 'address', 'sector', 'ceo', 'founded', 'founders', 'share_price', 'website', 'stock', 'area_served', 'revenue'])
    indexesToDelete = []
    for k in dict_matches.keys():
        indexesToDelete.append(k)
        row = df[df.index == k]
        for v in dict_matches[k]:
            indexesToDelete.append(v)
            for field in row:
                if(row[field].isnull().values.any()):
                    if(not df[df.index == v][field].isnull().values.any()):
                        row.at[k, field] = df[df.index==v][field].values[0]
        df_collapsed.loc[len(df_collapsed.index)]=row.values[0]
    return df_collapsed, indexesToDelete

df_collapsed_prima, indexesToDeleteGlobal = collapseMatches(getCompleteDataset(), dict_matches_global)
print('Tempo trascorso: '+ str(time.time() - start))
df_collapsed_prima.to_csv('output_integration/matches_collapsed.csv', index=False)

# Seconda passata di matching
#### criteri: matching 1:1 su name

In [ ]:
# questo dataset è già normalizzato
df_collapsed_prima = pd.read_csv('output_integration/matches_collapsed.csv')

# rimozione punti
df_collapsed_prima['name'] = df_collapsed_prima['name'].replace(r'\.', '', regex=True)

df_collapsed_prima_a = df_collapsed_prima
df_collapsed_prima_b = df_collapsed_prima

candidate_links = block('name', df_collapsed_prima_a, df_collapsed_prima_b)
multi_index = getMultiIndexNoCopy(candidate_links)

### Record Linkage solo su (name, name)

In [ ]:
compare = recordlinkage.Compare()

compare.string('name', 'name', method='jarowinkler', threshold=1)
# workaround
compare.string('country', 'country', method='jarowinkler', threshold=0)

# The comparison vectors
compare_vectors = compare.compute(candidate_links, df_collapsed_prima_a, df_collapsed_prima_b)
compare_vectors = compare_vectors[compare_vectors.index.isin(multi_index)]

matches = ecmFitPredict(compare_vectors)

dict_matches = getDictMatches(matches)

df_collapsed_seconda, indexesToDelete_seconda = collapseMatches(df_collapsed_prima, dict_matches)

In [ ]:
df_integrato_clean = df_collapsed_prima.drop(indexesToDelete_seconda)
df_collapsed_seconda = pd.concat([df_integrato_clean, df_collapsed_seconda], ignore_index=True)
print('Tempo trascorso: '+ str(time.time() - start))
df_collapsed_seconda.to_csv('output_integration/matches_collapsed_seconda.csv', index=False)

# Passo finale integration

### eliminazione righe 'doppione'
### concatenazione dataframe integrato dei match

In [ ]:
df = getCompleteDataset()
df_collapsed_seconda = pd.read_csv('output_integration/matches_collapsed_seconda.csv')

In [ ]:
df_clean = df.drop(indexesToDeleteGlobal)
df_final = pd.concat([df_clean, df_collapsed_seconda], ignore_index=True)
print('Tempo trascorso: '+ str(time.time() - start))
df_final.to_csv('output_integration/final_integration.csv', index=False)